In [2]:
import sys
sys.path.append("../..") 

from custom_scripts import database

In [6]:
### Table of unique carrier identifiers present in the test samples
carriers = database.query(""" 
                            SELECT op_unique_carrier, COUNT(*)
                                FROM flights_test
                                GROUP BY op_unique_carrier;
                            """)
carriers.to_csv('../../data/preprocessing/unique_carriers.csv', index=False)

In [7]:
### Table of flight numbers present in the test samples, and a count of their frequency
test_flight_numbers = database.query("""
                                SELECT op_carrier_fl_num, COUNT(*) 
                                    FROM flights_test
                                    GROUP BY op_carrier_fl_num;
                                """)
test_flight_numbers.to_csv('../../data/preprocessing/test_flight_numbers.csv', index=False)

In [8]:
### Table of flight numbers present in the samples available for training, and a count of their frequency
past_flight_numbers = database.query("""
                                SELECT op_carrier_fl_num, COUNT(*)
                                    FROM flights
                                    WHERE SUBSTRING(fl_date,6,2) = '01'
                                    GROUP BY op_carrier_fl_num;
                                """)
past_flight_numbers.to_csv('../../data/preprocessing/past_flight_numbers.csv', index=False)


In [10]:
### Historic average delay times by flight number
average_flight_delays = database.query("""
                                SELECT  op_carrier_fl_num, 
                                        AVG(carrier_delay), 
                                        AVG(weather_delay), 
                                        AVG(nas_delay), 
                                        AVG(late_aircraft_delay)
                                    FROM flights
                                    WHERE SUBSTRING(fl_date,6,2) = '01'
                                    GROUP BY op_carrier_fl_num;
                                """)
average_flight_delays.to_csv('../../data/preprocessing/average_flight_delays.csv', index=False)

In [16]:
flights = database.query("""
                                SELECT  fl_date, 
                                        op_unique_carrier, 
                                        tail_num, 
                                        op_carrier_fl_num, 
                                        origin_airport_id, 
                                        dest_airport_id, 
                                        crs_dep_time,
                                        crs_arr_time, 
                                        crs_elapsed_time, 
                                        dup, 
                                        flights, 
                                        distance
                                    FROM flights
                                    WHERE SUBSTRING(fl_date,6,2) = '01'
                                    ;
                                """)
flights

,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance
0,2019-01-03,MQ,N820AE,3607,15074,11298,1152,1304,72.0,N,1.0,225.0
1,2019-01-01,OO,N264SY,4613,11433,13930,2035,2111,96.0,N,1.0,235.0
2,2019-01-03,MQ,N248NN,3680,14027,11278,1450,1717,147.0,N,1.0,857.0
3,2019-01-01,OH,N575NN,5606,13577,11057,2157,2311,74.0,N,1.0,157.0
4,2019-01-01,OH,N254PS,5607,10529,11278,1151,1325,94.0,N,1.0,313.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1239863,2018-01-14,WN,N748SW,3835,14908,12191,1105,1605,180.0,N,1.0,1357.0
1239864,2018-01-14,WN,N917WN,3941,14908,12889,920,1025,65.0,N,1.0,226.0
1239865,2018-01-14,WN,N491WN,4330,14908,12889,1705,1805,60.0,N,1.0,226.0
1239866,2018-01-14,WN,N551WN,4539,14908,12889,2030,2130,60.0,N,1.0,226.0


In [8]:
import pandas as pd

import numpy as np

In [17]:
unique_carriers = pd.read_csv('../../data/preprocessing/unique_carriers.csv')
flight_numbers = pd.read_csv('../../data/preprocessing/test_flight_numbers.csv')
flights = flights[flights['op_unique_carrier'].isin(unique_carriers['op_unique_carrier'].values)]
flights[flights['op_carrier_fl_num'].isin(flight_numbers['op_carrier_fl_num'].values)]

,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance
0,2019-01-03,MQ,N820AE,3607,15074,11298,1152,1304,72.0,N,1.0,225.0
1,2019-01-01,OO,N264SY,4613,11433,13930,2035,2111,96.0,N,1.0,235.0
2,2019-01-03,MQ,N248NN,3680,14027,11278,1450,1717,147.0,N,1.0,857.0
3,2019-01-01,OH,N575NN,5606,13577,11057,2157,2311,74.0,N,1.0,157.0
4,2019-01-01,OH,N254PS,5607,10529,11278,1151,1325,94.0,N,1.0,313.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1239863,2018-01-14,WN,N748SW,3835,14908,12191,1105,1605,180.0,N,1.0,1357.0
1239864,2018-01-14,WN,N917WN,3941,14908,12889,920,1025,65.0,N,1.0,226.0
1239865,2018-01-14,WN,N491WN,4330,14908,12889,1705,1805,60.0,N,1.0,226.0
1239866,2018-01-14,WN,N551WN,4539,14908,12889,2030,2130,60.0,N,1.0,226.0
